In [ ]:
'''
Nonograms is a puzzle that involves filling a 2D grid with black or white pixels to make a picture
The goal of this program is to solve a Nonogram given the clues on the rows and columns.
Note that solving Nonograms is NP-complete.
You probably will need to install PIL, a Python Imaging Library
'''

In [35]:
# Imaging library: creating images and editing a pixel color
from PIL import Image
import math
import time

In [3]:
# square size relative size
square_size = 3

# actual size in pixels; I want a multiple of 8
p = 8 * square_size

# common colors
white = (255, 255, 255, 255)
black = (0, 0, 0, 255)
red = (255, 0, 0, 255)
green = (0, 255, 0, 255)
blue = (0, 0, 255, 255)

# define a p x p square of types found on board
white_square = [[white for i in range(p)] for j in range(p)]
black_square = [[black for i in range(p)] for j in range(p)]
x_square = [[white for i in range(p)] for j in range(p)]
question_square = [[white for i in range(p)] for j in range(p)]

# building an x
for i in range(square_size, p-square_size):
    x_square[i][i] = red
    x_square[p-i][i] = red

# implement question mark, tho as the transpose
question_coords = ((2, 2), (3, 1), (4, 1), (5, 2), (5, 3), (4, 4), (3, 5), (3, 7))

for coord in question_coords:
    for i in range(square_size):
        for j in range(square_size):
            question_square[coord[0] * square_size + i][coord[1] * square_size + j] = black

In [4]:
# Square represents a single square on the nonogram board
class Square():
    white_mark = 0
    black_mark = 1
    x_mark = 2
    question_mark = 3
    
    def __init__(self, row, col):
        # position of the Square globally
        self.row = row
        self.col = col
        
        # in case you make a guess, so mark which tentatively came first
        self.tentative_number = 0
        
        # the current Square status
        self.mark = Square.question_mark
        # if black, what position in row_num black square associated with
        self.assoc_index = -1
        # if black, what length in row_num black square associated with
        self.assoc_len = -1
        # insert time
        self.insert_time = -1
        
    def is_finished(self):
        return self.finished
    
    def get_mark(self):
        return self.mark
    
    def is_white(self):
        return self.mark == Square.white_mark
    
    def is_black(self):
        return self.mark == Square.black_mark
    
    def is_x(self):
        return self.mark == Square.x_mark
    
    def is_question(self):
        return self.mark == Square.question_mark
        
    def set_white(self):
        self.color = white
        self.mark = Square.white_mark
        self.finished = True
        
    def set_black(self):
        self.color = black
        self.mark = Square.black_mark
        self.finished = True
        
    def set_x(self):
        self.mark = Square.x_mark
        self.finished = True
        
    def set_question_mark(self):
        self.mark = Square.question_mark
        
    def set_mark(self, input_mark):
        if input_mark == Square.white_mark:
            self.set_white()
        elif input_mark == Square.black_mark:
            self.set_black()
        elif input_mark == Square.x_mark:
            self.set_x()
        elif input_mark == Square.question_mark:
            self.set_question_mark()

In [5]:
class WorkingSet():
    
    def __init__(self, row, col, left, right, clue_nums, clue_lens, certainty):
        # left range, right range, clues associated with this working set, and certainty factor of each clue
        # certainty factor 0 for not sure, 1 for certain (won't appear in another working set)
        # -1 for certain it should not be a clue in this working set
        self.row = row
        self.col = col
        
        self.left = left
        self.right = right
        
        self.clue_nums = clue_nums
        self.clue_lens = clue_lens
        self.certainty = certainty
        
    

In [27]:
# Board represents the entire matrix of Squares, tho currently solve is outside of class Board
class Board():
    
    def __init__(self, R, C, row_nums, col_nums):
        # size of board
        self.R = R
        self.C = C
        
        # take in row and column clues
        self.row_nums = row_nums
        self.col_nums = col_nums
        
        # build the actual matrix and its transpose
        self.board = [[Square(i, j) for j in range(C)] for i in range(R)]
        self.transpose = [[self.board[i][j] for i in range(R)] for j in range(C)]
        
        # build the working_sets: intervals on each row to say what is not yet filled, and corresponding clues
        self.row_ws = {}
        for i in range(R):
            self.row_ws[i] = [WorkingSet(i, -1, 0, C-1, [j for j in range(len(row_nums[i]))], row_nums[i], [1 for j in range(len(row_nums[i]))])]
        
        self.col_ws = {}
        for i in range(C):
            self.col_ws[i] = [WorkingSet(-1, i, 0, R-1, [j for j in range(len(col_nums[i]))], col_nums[i], [1 for j in range(len(col_nums[i]))])]
    
        self.assoc_list_row = [[[-2] for j in range(C)] for i in range(R)]
        self.assoc_list_col = [[[-2] for j in range(C)] for i in range(R)]
    
    # mode = 0: board, no lines in between, in surrounding frame
    # mode = 1: thickness lines between each squares
    # mode = 2: split bigger squares from off each other
    def draw(self, mode=0):
        # buffer is the size of the surrounding frame of the matrix when drawing
        # how wide margins are in pixels
        buffer = 10
        
        # how wide of a space between each square
        thickness = 0
        
        # how many consecutive squares before having a larger gap
        alt = 0
        # larger gap
        alt_thickness = 0
        
        if mode == 1:
            thickness = 1
        elif mode == 2:
            thickness = 1
            alt = 5
            alt_thickness = 3
        
        # calculate the height and width of the image to render
        height = buffer + p * self.R + thickness * (self.R - 1) + buffer
        width = buffer + p * self.C + thickness * (self.C - 1) + buffer
        
        if alt != 0:
            num_extra_R = (self.R - 1) // alt
            num_extra_C = (self.C - 1) // alt
            
            height += (num_extra_R * (alt_thickness - thickness))
            width += (num_extra_C * (alt_thickness - thickness))
        
        # Create new image, load matrix of pixels
        img = Image.new("RGBA", (width,height,))
        pixels = img.load()
        
        # fill background (and buffer / frame) green
        for i in range(height):
            for j in range(width):
                pixels[j,i] = green
                
        indexr = 0
        indexc = 0
        
        row = buffer
        col = buffer
        
        # fill each small square in the matrix of Squares
        while indexr < self.R:
            if indexc == self.C:
                indexc = 0
                indexr += 1
                
                row += p
                row += thickness
                if alt != 0 and (indexr + alt - 1) % alt == alt - 1:
                    row += (alt_thickness - thickness)
                
                col = buffer
                continue
            
            pane = question_square
            s = self.board[indexr][indexc]
            if s.is_white():
                pane = white_square
            elif s.is_black():
                pane = black_square
            elif s.is_x():
                pane = x_square
            
            for i in range(p):
                for j in range(p):
                    pixels[col + i, row + j] = pane[i][j]
            
            indexc += 1
            col += p
            col += thickness
            if alt != 0 and (indexc + alt - 1) % alt == alt - 1:
                col += (alt_thickness - thickness)
        
        # save image and display
        img.save("output2.png")
        img.show()
    
    # display statistics of board for debugging purposes
    def show_stats(self):
        N = self.R
        M = self.C
        num_black = 0
        num_x = 0
        num_question = 0
        for i in range(N):
            for j in range(M):
                s = self.board[i][j]
                if s.get_mark() == Square.black_mark:
                    num_black += 1
                elif s.get_mark() == Square.x_mark:
                    num_x += 1
                elif s.get_mark() == Square.question_mark:
                    num_question += 1
                    
        log2 = num_question * math.log(2, 10)

        print("height =", N, "width =", M)
        print("Black:", num_black, ", X:", num_x, ", ?:", num_question)
        print("Unknown:", num_question, "/", N*M)
        print("Brute force has at most 2 ^", num_question, "~", pow(10, log2-math.floor(log2)), "* 10 ^", int(math.floor(log2)), "states")


In [28]:
# put in exactly row_num black squares in a row, at position pos or later
# O(len(row)s)
def fill_in_earliest(row, row_num, pos):
    N = len(row)
    if pos < 0 or pos >= N:
        return (-1, -1)
    
    pos1 = pos
    
    while 1:
        while pos1 < N and row[pos1].is_x():
            pos1 += 1

        if pos1 >= N:
            return (-1, -1)

        # now pos1 is at a black mark or question mark
        pos2 = pos1
        while pos2 < N and (not row[pos2].is_x()) and pos2 - pos1 + 1 < row_num:
            pos2 += 1

        # either pos2 == N or row[pos2] is X or pos2-pos1+1 == row_num
        if pos2 == N:
            return (-1, -1)
        if row[pos2].is_x(): # reset first pointer at right pointer + 1 and restart
            pos1 = pos2 + 1
            continue
            
        # must be true that pos2 - pos1 + 1 == row_num: right size!
        # but what if there are black squares immediately before or after it?
        
        if 0 < pos1 and row[pos1-1].is_black():
            #pos1 = pos2 + 1
            pos1 = pos1 + 1
            continue
            
        if pos2 < N-1 and row[pos2+1].is_black():
            #pos1 = pos2 + 1
            
            # inefficient!  but more correct than above :(
            pos1 = pos1 + 1
            continue
        
        for j in range(pos1, pos2+1):
            row[j].set_mark(Square.black_mark)
            
        return (pos1, pos2)

In [29]:

def apply_early_late_row(ws, b):
    '''
    Takes in a working set, fills in board
    returns list of smaller working sets
    '''
    N = ws.right - ws.left + 1
    
    board = b.board
    transpose = b.transpose
    
    if N <= 0:
        return []
    
    row = None
    if ws.row == -1:
        row = transpose[ws.col]
    else:
        row = board[ws.row]
    
    # make a copy of the row for earliest
    earliest = [Square(0, 0) for i in range(N)]
    latest_reverse = [Square(0, 0) for i in range(N)]
    for i in range(ws.left, ws.right+1):
        earliest[i - ws.left].set_mark(row[i].get_mark())
        latest_reverse[N-1-i+ws.left].set_mark(row[i].get_mark())
        
    clue_numE = [-1 for i in range(N)]
    clue_numR = [-1 for i in range(N)]
    
    index = 0
    starting_pos = 0
    while index < len(ws.clue_nums) and starting_pos <= ws.right - ws.left:
        pos1, pos2 = fill_in_earliest(earliest, ws.clue_lens[index], starting_pos)
        
        if pos1 == -1:
            # invalid clue if you can't place into earliest interpretation
            which_clue = ws.clue_nums[index]
            for j in range(index, len(ws.clue_nums)):
                ws.certainty[j] = -1
            
            # ?? how to update certainty of other working set to 1?
            
            print("bruhhhhhhhhhhhhhhhh how did you get here; invalid clue")
            
            return []
        
        for j in range(pos1, pos2 + 1):
            clue_numE[j] = index
        
        index += 1
        starting_pos = pos2 + 1
    
    index = len(ws.clue_nums) - 1
    starting_pos = 0
    while index >= 0 and starting_pos <= ws.right - ws.left:
        pos1, pos2 = fill_in_earliest(latest_reverse, ws.clue_lens[index], starting_pos)
        
        if pos1 == -1:
            print("still have no idea how you got here")
            return []
        
        for j in range(pos1, pos2 + 1):
            clue_numR[j] = index
        
        index -= 1
        starting_pos = pos2 + 1
        
    latest = [latest_reverse[i] for i in range(len(latest_reverse) - 1, -1, -1)]
    clue_numL = [clue_numR[i] for i in range(N-1, -1, -1)]
    
    # shift back information from earliest and latest back into board
    
    for i in range(N):
        if clue_numE[i] >= 0 and clue_numE[i] == clue_numL[i]:
            if ws.row == -1:
                # column
                board[i + ws.left][ws.col].set_mark(Square.black_mark)
                b.assoc_list_col[i+ws.left][ws.col] = [clue_numE[i]]
            else:
                # row
                board[ws.row][i + ws.left].set_mark(Square.black_mark)
                b.assoc_list_row[ws.row][i + ws.left] = [clue_numE[i]]
    

def apply_early_late(b):
    '''
    append list of smaller working sets to modify original
    '''
    
    for k, v in b.row_ws.items():
        # k is row number, v is list of working sets
        for ws in v:
            # ws is a working set
            apply_early_late_row(ws, b)
           
    for k, v in b.col_ws.items():
        # k is row number, v is list of working sets
        for ws in v:
            # ws is a working set
            apply_early_late_row(ws, b)
    

In [ ]:
def update_assoc_lists_row(N, row, row_nums, b, assoc_list):
    # have simplistic lower and upper bounds
    # "if this were clue 3, then the rest can't fit" kind of reasoning
    
    num_clues = len(row_nums)
    
    # for each square, get earliest and latest possible clue numbers they can be associated with
    
    earliest_clues = []
    if assoc_list[0][0] == -2:
        earliest_clues.append(0)
    else:
        earliest_clues.append(assoc_list[0][0])
        
    for i in range(1, N):
        temp = assoc_list[i][0]
        if temp == -2:
            temp = 0
        earliest_clues.append(max(earliest_clues[-1], temp))
    
    latest_clues_reversed = []
    if assoc_list[N-1][0] == -2:
        latest_clues_reversed.append(N-1)
    else:
        latest_clues_reversed.append(assoc_list[N-1][0])
        
    for i in range(N-2, -1, -1):
        temp = assoc_list[i][0]
        if temp == -2:
            temp = N-1
        latest_clues_reversed.append(min(latest_clues_reversed[-1], temp))
    
    latest_clues = [latest_clues_reversed[i] for i in range(N-1, -1, -1)]
    
    # for each square, if X, continue; 
    # if there's only possible clue for this square, set it as such and continue
    # if multiple, try to cross some out by contradiction
    
    for i in range(N):
        s = row[i]
        if s.is_x():
            assoc_list[i] = [-1]
            continue
        elif 
    

def update_assoc_lists(b):
    # run on all rows, columns; each independent

In [30]:
def solve(b):
    R = b.R
    C = b.C
    N = R
    M = C
    
    board = b.board
    
    apply_early_late(b)

In [31]:
# deal with input from "input.txt": read in size and row/col numbers
# Format: first line has 2 numbers: number of rows, then number of columns
# next |R| lines: the numbers on the rows, from left to right
# next |C| lines: the numbers on the columns, from top to bottom
def get_input(file_name="input.txt"):
    with open(file_name, 'r') as f:
        N, M = [int(x) for x in f.readline().split()]
        
        row_nums = []
        for i in range(N):
            temp = [int(x) for x in f.readline().split()]
            row_nums.append(temp)
            
        col_nums = []
        for i in range(M):
            temp = [int(x) for x in f.readline().split()]
            col_nums.append(temp)
            
        return row_nums, col_nums

In [36]:
# first function to be run
def main():
    # read in input
    row_nums, col_nums = get_input("input.txt")
    
    # get size of board
    R = len(row_nums)
    C = len(col_nums)
    
    start_time = time.time()
    
    # create board
    b = Board(R, C, row_nums, col_nums)
    
    # solve board
    solve(b)
    
    end_time = time.time()
    
    # render and show the board
    b.draw(2)
    
    for i in range(R):
        for j in range(C):
            print(b.assoc_list_row[i][j], ", ", end="")
        print("")
    
    print("columns:")
    for i in range(R):
        for j in range(C):
            print(b.assoc_list_col[i][j], ", ", end="")
        print("")
    
    # display statistics of board
    b.show_stats()
    
    print("Solving took", end_time - start_time, " seconds")

In [37]:
if __name__ == '__main__':
    main()

[-2] , [-2] , [0] , [0] , [0] , [0] , [0] , [0] , [0] , [0] , [-2] , [-2] , [-2] , [-2] , [-2] , 
[-2] , [-2] , [-2] , [-2] , [0] , [0] , [0] , [0] , [0] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , 
[-2] , [0] , [0] , [0] , [0] , [0] , [0] , [0] , [0] , [-2] , [-2] , [1] , [1] , [1] , [-2] , 
[0] , [0] , [0] , [0] , [0] , [0] , [0] , [0] , [0] , [-2] , [1] , [1] , [1] , [1] , [1] , 
[-2] , [-2] , [-2] , [-2] , [-2] , [1] , [1] , [1] , [-2] , [-2] , [-2] , [2] , [2] , [-2] , [-2] , 
[-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , 
[-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , 
[-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , 
[-2] , [-2] , [0] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , 
[-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] , [-2] ,

In [ ]:
# Input Test Cases!
'''
Possible input in input.txt:
****************************
5 5
5
1
5
1
5
3 1
1 1 1
1 1 1
1 1 1
1 3

****************************

5 5
3
2 2
1 1
2 2
3
3
2 2
1 1
2 2
3

****************************

5 5
5
1 1 1
5
1
1
5
1 1
3
1 1
3

****************************

5 5
5
1
1
1
5
1 1
1 2
1 1 1
2 1
1 1

****************************
Swan
15 15
10 2
9 1
9 4
9 5
2 5 4
1 1 2 4
2 2 2 3
2 1 2 2 2
3 2 1 2 1
3 2 1 1 1
3 2 1 1 1
2 2 1 1 1
1 1 1
2 2
15
4 6 3
5 6 2
5 4 1
4 1 1 1
4 1 1 1 1
5 1 1 1 1
5 1 1 1 1
5 1 1 1 1
6 1 1 1
1 4 1 1 1
1 3 1
4 1 1
5 4 1
1 6 2
15

****************************
Troll Face
40 50
23
19 6
8 3
3 2
3 7 9 2
3 2 4 2 2
2 2
1 1
1 6 2
2 4 10 2
2 7 2 8 1 3
2 2 10 2 10 1 3
5 4 4 2 6 2
2 4 1 2 4 3 2
2 8 2 6 1 2 2
2 1 4 2 3 2 1
2 1 2 4 7 2 1
2 3 4 1 2 4 6 2
4 3 5 4 1 5 2 1 2
1 4 4 2 8 3 3
2 6 2 5 7 3
2 23 7 3
1 5 12 2 6 2 2
1 4 2 2 1 9 2 2
1 26 4 1
1 24 2 2
1 20 2 3 1
1 17 2 3 2
2 5 2 1 1 4 2
2 5 2 2 1 5 1 2
2 22 2 2 3
2 16 4 3
2 4 4
2 1 5 4
2 2 5 4
2 4 4
2 4
2 6
2 8
16
5
8
3 5 9
2 1 1 18
5 2 1 2
3 1 2 1 2
2 211 1 1
2 17 1 1
2 2 14 2 1
2 2 12 1 1
1 2 2 2 7 1 1
2 3 2 12 1 1
2 3 2 1 7 2 1
2 3 2 2 8 1
2 3 2 2 8 1
2 2 3 2 4 2 1
2 1 3 5 7 2 1
2 2 5 13 1
2 1 2 1 4 4 3 2
2 1 2 1 2 4 2 2
2 1 2 4 2 2
2 1 1 7 2 1
2 1 1 2 8 1 2 
2 2 1 2 1 3 2 1 2
2 1 3 1 1 1 3 2 1 2
2 1 2 1 1 3 3 2 2 2 
2 2 1 2 6 2 2 2 2
2 1 2 2 2 6 2 1 2
2 1 4 9 2 2 2
2 1 4 2 8 2 2
1 1 5 1 3 2 2 2
1 1 6 2 3 2 2 2
1 1 4 2 2 2 2 2 2
1 1 3 1 2 3 2 2 2
2 2 3 1 8 2 2
2 1 2 1 10 1 2
2 2 2 2 2 3
2 2 2 2 5 1 2
2 2 1 5 2
2 2 2 3 1
2 1 7 2
3 1 4 2
3 2 2 2
4 2 1 3
3 6 3
3 4 3
2 4
2 3
3 4
6

'''